# Bag of Words in Action dari Dataset Wayang

M. Imam Iryunullah && Muhammad Azmi Nur Iman

In [1]:
import json
import pandas as pd
import numpy as np
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords

# Ngeload Dataset Wayang

In [2]:
with open("output/pandawa_dataset_v2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
# Mengambil isi data pada json pada data answer

In [3]:
korpus = pd.Series([item["answer"] for item in data])

In [4]:
pabrik = StemmerFactory()
stemmer = pabrik.create_stemmer()
stop_kata = set(stopwords.words("indonesian"))

In [5]:
def pra_proses(korpus, daftar_tetap=None, stemming=True, jenis_stem="sastrawi",
               lemmatisasi=False, hapus_stopword=True):
    hasil = []
    
    for teks in korpus:
        teks = teks.lower()

        if daftar_tetap:
            for w in daftar_tetap:
                teks = teks.replace(w.lower(), w.replace(".", ""))  

        teks = re.sub(r"\d+", "", teks)
        teks = teks.translate(str.maketrans("", "", string.punctuation))

        token = teks.split()

        if hapus_stopword:
            token = [t for t in token if t not in stop_kata]
        if stemming and jenis_stem == "sastrawi":
            token = [stemmer.stem(t) for t in token]
        
        if daftar_tetap:
            for w in daftar_tetap:
                token = [t if t != w.replace(".", "").lower() else w for t in token]
        
        hasil.append(" ".join(token))
    
    return hasil

In [6]:
kata_tetap = ['U.S.', 'Mr.', 'Mrs.', 'D.C.']

In [7]:
pra_korpus = pra_proses(
    korpus,
    daftar_tetap=kata_tetap,
    stemming=True,          
    jenis_stem="sastrawi",     
    lemmatisasi=False,       
    hapus_stopword=True
)


# Kosa Kata

In [8]:
kosakata_set = set()
for kalimat in pra_korpus:
    for kata in kalimat.split():
        kosakata_set.add(kata)

kosakata = list(kosakata_set)

Buat Posisi Index

In [46]:
posisi = {}
for i, token in enumerate(kosakata):
    posisi[token] = i


# Matrix Bow

In [47]:
bow_matriks = np.zeros((len(pra_korpus), len(kosakata)))

for i, kalimat in enumerate(pra_korpus):
    for kata in kalimat.split():
        bow_matriks[i][posisi[kata]] += 1

--- HASIL ---

In [48]:
print("Jumlah dokumen :", bow_matriks.shape[0])
print("Ukuran kosakata:", bow_matriks.shape[1])
print("Contoh matrix:\n", bow_matriks[:5])

Jumlah dokumen : 33
Ukuran kosakata: 2915
Contoh matrix:
 [[ 0. 21.  0. ...  2.  0.  0.]
 [ 0.  0.  2. ...  4.  0.  0.]
 [ 0.  0.  0. ...  2.  0.  1.]
 [ 0.  0.  0. ...  8.  0.  0.]
 [ 0.  0.  1. ...  2.  0.  0.]]
